In [1]:
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import shlex
import json
import cProfile, pstats, sys
import cv2
import sys
import ast
import csv
import pickle
import re
from modules.path_planning.astar import AStar
from modules.path_planning.RRA import RRA
from modules.path_planning.FIXED_ARA import ARA
from modules.path_planning.DLITE import DLITESEARCH
from modules.path_planning.RRT_star import RRT_Star
import time
import random
import math
import pandas as pd
import os
import copy
from os.path import isfile,join
static_or_dynamic = input("Input STATIC for static path planning. Anything else for dynamic: ")
pathPlannerNames = None
temp_file_name = input("Input path to output image files: ")

if static_or_dynamic == "STATIC":
    pathPlannerNames = ['RRT*', 'astar', 'ASTARTHRESHOLD', 'ARA']
    if not os.path.exists(temp_file_name+"\STATIC"):
        os.makedirs(temp_file_name+"\STATIC\RRT\generated_path_")
        os.makedirs(temp_file_name+"\STATIC\ASTAR\generated_path_")
        os.makedirs(temp_file_name+"\STATIC\ASTARTHRESHOLD\generated_path_")
        os.makedirs(temp_file_name+"\STATIC\ARA\generated_path_")
else:
    pathPlannerNames = ['RRA*', 'D* Lite', 'ARA']
    if not os.path.exists(temp_file_name+"\DYNAMIC"):
        os.makedirs(temp_file_name+'\DYNAMIC\RRA\generated_path_')
        os.makedirs(temp_file_name+'\DYNAMIC\ARA\generated_path_')
        os.makedirs(temp_file_name+'\DYNAMIC\DSTARLITE\generated_path_')
    
fieldnames = ['Image Name', 'Image Size', 'Start Position', 'Goal Position', 'Success?']
fieldnames.extend(pathPlannerNames)
#csvFileName = 'dump_output.csv'

csvFileName = input("Output CSV file path: ")
alreadyGenerated = input("Input False if metrics have already been generated.")
pathMetrics = []
thewriter = None

   # thewriter.writerow({'Image Name': 'sample_pred0', 'Start Position': (15, 15), 'Goal Position': (19,19) })

startX, startY, endX, endY = 0,0,0,0
selectStart = True
selectEnd = True
img = None
im_Size = 600
img1 = None
prediction_matrix = None
path = None
path_runner = None

img_copy = None
ground_truth_im_copy = None
filename = None


def runPathPlanning(dynamic_or_static):
    global startX, startY, endX, endY
    startPos = (startX,startY)
    endPos = (endX,endY)
    global path
    global alreadyGenerated
    global path_runner
    global img, img1
    global temp_file_name
    global prediction_matrix
    global filename
    global pathPlannerNames
    algoPlanner_information = []
    prediction_matrix_copy = copy.deepcopy(prediction_matrix)
    img_copy = copy.deepcopy(img)
    ground_truth_im_copy = copy.deepcopy(img1)
    
    for algoName in pathPlannerNames:
        l = temp_file_name
        start = time.time()

        if algoName == "astar":
            path_runner = AStar(startPos,endPos,copy.deepcopy(img),"image", prediction_matrix)
            path = path_runner.run()
        
        elif algoName == "ASTARTHRESHOLD":
            im_c = copy.deepcopy(img)
            for p in range(len(prediction_matrix)):
                for z in range(len(prediction_matrix)):
                    if prediction_matrix[p][z][1] >= 0.30:
                        im_c[p][z] = [255, 255, 255]
                    else:
                        im_c[p][z] = [0,0,0]
                        
            path_runner = AStar(startPos,endPos,im_c,"image", prediction_matrix)
            path = path_runner.run()
                    

        elif algoName == "ARA":
            path_runner = ARA(startPos, endPos, img, prediction_matrix, 0.75, img1)
            path = path_runner.static_path_main()

        elif algoName == "RRT_Star" or algoName == "RRT*":
            path_runner = RRT_Star(startPos, endPos, 5, 200, 50, copy.deepcopy(img))
            path = path_runner.run()

        elif algoName == "RRA*":
            path_runner = RRASEARCHTREE(startPos, endPos, img, img1)
            path = path_runner.ComputePath()

        elif algoName == "D* LITE":
            path_runner = DLITESEARCH(startPos, endPos, img, img1, prediction_matrix)
            path = path_runner.DLITERUN()

        else:
            print("No valid algorithm found for the algorithm name given...exiting")
            exit()

        end = time.time()
        duration = end - start
        solution_quality = 0.0
        goal_found = False

        similarity_Score = GTsimilarity(path, img1)

         
        if path != None and endPos in path:
            map_path = []
            first_point = path[0]
            second_point = None
            for pos in path:
                map_path.append(pos)
                if pos != first_point:
                    second_point = pos
                    cv2.line(img1, first_point, second_point, color=(0, 255, 0), thickness=5)
                    cv2.line(img, first_point, second_point, color=(0, 255, 0), thickness=5)

                    first_point = pos
                if pos != path[0]:
                    solution_quality += math.hypot((map_path[-1][0] - map_path[-2][0]),(map_path[-1][1] - map_path[-2][1]))
            goal_found = True
        else: 
            if path == None or (endX, endY) not in path:
                solution_quality = np.inf
            else:
                map_path = []
                second_point = None
                first_point = path[0]
                for pos in path:
                    map_path.append(pos)
                    if pos != path[0]:
                        solution_quality += math.hypot((map_path[-1][0] - map_path[-2][0]),(map_path[-1][1] - map_path[-2][1]))
                    
                
                    
            
        csvCellHolder = None
        if dynamic_or_static == "STATIC":
            csvCellHolder = [duration, solution_quality / np.linalg.norm(np.array([startX, startY])-np.array([endX, endY])), similarity_Score, goal_found]
        else:
            csvCellHolder = [duration, solution_quality / np.linalg.norm(np.array([startX, startY])-np.array([endX, endY])), path_runner.replans]

        img1 = cv2.circle(img1, (startX, startY), radius=5, color=(255, 0, 0), thickness=-1)
        img1 = cv2.circle(img1, (endX, endY), radius=5, color=(0, 0, 255), thickness=-1)
        img = cv2.circle(img, (startX, startY), radius=5, color=(255, 0, 0), thickness=-1)
        img = cv2.circle(img, (endX, endY), radius=5, color=(0, 0, 255), thickness=-1)
        
        if dynamic_or_static == "STATIC":
        
            if algoName == "RRT_Star" or algoName == "RRT*":
                l += r'\STATIC\RRT\generated_path_' + filename    
            elif algoName == "ARA":
                l += r'\STATIC\ARA\generated_path_' + filename
            elif algoName == "astar":
                l += r'\STATIC\ASTAR\generated_path_' + filename
            elif algoName == "ASTARTHRESHOLD":
                l += r'\STATIC\ASTARTHRESHOLD\generated_path_' + filename
                
        else: 
 
            if algoName == "D* Lite":
                l += r'\DYNAMIC\DSTARLITE\generated_path_' + filename    
            elif algoName == "ARA":
                l += r'\DYNAMIC\ARA\generated_path_' + filename
            elif algoName == "RRA*":
                l += r'\DYNAMIC\RRA\generated_path_' + filename
            
            
            
        cv2.imwrite(l+'.jpg', img1)
        cv2.imwrite(l+'segmentated'+'.jpg', img)
        
        algoPlanner_information.append(csvCellHolder)

        img1 = copy.deepcopy(ground_truth_im_copy)
        img = copy.deepcopy(img_copy)
        prediction_matrix = copy.deepcopy(prediction_matrix_copy)
    return algoPlanner_information

def GTsimilarity(path, gt_img):
    if path == None:
        return 0.0
    ratio = 0
    for a in path:
        if np.array_equal(gt_img[a[1]][a[0]], [255, 255, 255]):
            ratio += 1

    return (ratio/len(path)) * 100

def main():
    os.environ["CUDA_VISIBLE_DEVICES"] = "1"

    global img 
    global img1
    global prediction_matrix
    global im_Size
    global filename
    global img_copy
    global ground_truth_im_copy, static_or_dynamic
    global thewriter
    global alreadyGenerated
    global startX, startY, endX, endY
    global fieldnames
    
    global csvFileName
    calculate_ret_array = []
    if alreadyGenerated == 'True':
        
        with open(csvFileName, 'w+') as csvfile:
            thewriter = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter = ';')
            thewriter.writeheader()

            imgFolder = input("Path to Image Folder: ")
           # groundTruthFolder = "C:/Users/charles/Downloads/sample_predictions3/ground_truth"
            groundTruthFolder = input("Path to ground Truth folder: ")
           # predictionFolder = input("Path to prediction folder: ")
            predictionMatrixFolder = input("Path to prediction matrix folder. End with a /: ")

            appender = []
            points_input = input("Path to points csv file: ")
            
            with open(points_input, 'r') as file:
                csvreader = csv.reader(file, delimiter=";")
                for row in csvreader:
                    if len(row) == 0:
                        continue
                    appender.append(row)

            for point_arrays in appender:
                t = shlex.split(point_arrays[0])
                print(t)
                filename = t[0]
                print(filename)
                with open(predictionMatrixFolder+filename[0:-3]+'pkl', 'rb') as f:
                    prediction_matrix = pickle.load(f)
                img = cv2.resize(cv2.imread(join(imgFolder, filename)), (im_Size, im_Size))
                img1 = cv2.resize(cv2.imread(join(groundTruthFolder, filename)), (im_Size, im_Size))
                (startX, startY) =  eval(t[1])
                (endX, endY) =  eval(t[2])
                prediction_matrix = cv2.resize(prediction_matrix, (im_Size, im_Size))
                return_array = runPathPlanning(static_or_dynamic)
                calculate_ret_array.append(return_array)
                thewriter.writerow({'Image Name': filename, 'Image Size': im_Size, 'Start Position': (startX, startY), 'Goal Position': (endX, endY), pathPlannerNames[0]: return_array[0] , pathPlannerNames[1]: return_array[1], pathPlannerNames[2]: return_array[2], pathPlannerNames[3]: return_array[3]})


    
    
    
    
    data = pd.read_csv(csvFileName, delimiter=';')
    head_data = []
    success_rate = [0] * len(pathPlannerNames)
    avg_path_acc = [0.0] * len(pathPlannerNames)
    norm_path_length = [0.0] * len(pathPlannerNames)

        

    print(avg_path_acc)
    
    #RRt*, astar, astarthresh, ara
    constant = -1
        
        
    with open(csvFileName) as csv_file:

        # creating an object of csv reader
        # with the delimiter as ,
        csv_reader = csv.reader(csv_file, delimiter = ',')

        # list to store the names of columns

        # loop to iterate through the rows of csv
        len_iter = 0
        for row in csv_reader:
            arow = re.split(';+', ''.join(row))[4:]
            if len(row) == 0:
                continue
            elif constant == -1:
                constant += 1
                len_iter = len(arow[1:])
                continue
            constant += 1
            
            # adding the first row
            arow = [s.replace(' ', ',').strip('][').split(',') for s in arow]
            tmp_val = getMaxValInList(arow)
            for a in range(len_iter):
                if arow[a][3] == 'True':
                    success_rate[a] += 1
                avg_path_acc[a] += float(arow[a][2])
                if arow[a][1] == 'inf' or (arow[a][1] == '0.0' and arow[a][3] == 'False'):
                    norm_path_length[a] += 2 * tmp_val
                else:
                    norm_path_length[a] += float(arow[a][1])
                    

        print(success_rate)
        for z in range(len(pathPlannerNames)):
            success_rate[z] /= constant
            norm_path_length[z] /= constant
            avg_path_acc[z] /= constant
            print(pathPlannerNames[z] + ':', [success_rate[z] * 100, norm_path_length[z], avg_path_acc[z]])
                    
                
                    


 

    #success rate
    #path acc
    #
    
def getMaxValInList(given_list):
    t = 0.0
    for a in range(len(given_list)):
        if float(given_list[a][1]) > t and given_list[a][1] != 'inf':
            t = float(given_list[a][1])
            
    return t
            
    
    
    
if __name__ == "__main__":
    main()



Input STATIC for static path planning. Anything else for dynamic: STATIC
Input path to output image files: DeepGlobePaths
Output CSV file path: deepglobe.csv
Input False if metrics have already been generated.True
Path to Image Folder: C:/Users/charles/Downloads/Filtered_Ensemble_DeepGlobe/Prediction
Path to ground Truth folder: C:/Users/charles/Downloads/Filtered_Ensemble_DeepGlobe/Ground Truth
Path to prediction matrix folder. End with a /: C:/Users/charles/Downloads/Filtered_Ensemble_DeepGlobe/Prediction Matrix/
Path to points csv file: deepglobepoints.csv
['sample_pred_0.png', '(77, 389)', '(472, 373)']
sample_pred_0.png
57
Found the goal after  24468  node expansions
Found a solution!
Found the goal after  24472  node expansions
Found a solution!
Beginning ARA* search from (77, 389) to (472, 373)
[[0.99685276]
 [0.00639719]
 [0.5135652 ]
 [0.7858267 ]
 [0.22887427]]
Initializing tree:
ARA* initial path has an accuracy of:  95.70707070707071 %
['sample_pred_137.png', '(32, 406)', '

Found the goal after  35792  node expansions
Found a solution!
Beginning ARA* search from (78, 112) to (497, 560)
[[0.9962517 ]
 [0.00950927]
 [0.521618  ]
 [0.7956737 ]
 [0.24020815]]
Initializing tree:
ARA* initial path has an accuracy of:  25.134649910233392 %
['sample_pred_85.png', '(570, 186)', '(536, 416)']
sample_pred_85.png
21
Found the goal after  4075  node expansions
Found a solution!
Found the goal after  4080  node expansions
Found a solution!
Beginning ARA* search from (570, 186) to (536, 416)
[[0.99653846]
 [0.00742662]
 [0.51701844]
 [0.7927605 ]
 [0.2380932 ]]
Initializing tree:
ARA* initial path has an accuracy of:  92.64069264069265 %
['sample_pred_86.png', '(244, 72)', '(292, 406)']
sample_pred_86.png
1
Beginning ARA* search from (244, 72) to (292, 406)
[[0.9965992 ]
 [0.01212138]
 [0.527058  ]
 [0.24268717]
 [0.7974589 ]]
Initializing tree:
ARA* initial path has an accuracy of:  48.00693240901213 %
['sample_pred_9.png', '(30, 495)', '(343, 367)']
sample_pred_9.png
